[View in Colaboratory](https://colab.research.google.com/github/choudharykrish/ContentBlocker/blob/master/IntegrationWithDocs.ipynb)

In [0]:
!kill -9 -1

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

# Mounting Drive

In [2]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [4]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force

Enter verification code: ··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [5]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

#print 'Files in Drive:'
!ls drive/

# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/created.txt

199d0f91-feb7-4657-a1c8-9f3b3512503d-large.jpeg  created.txt
5th Sem						 EYRC
6th Sem						 HackWithInfy
Advanced Java CPD				 HackWithInfy Travel Documents
Certifications Originals - Page 5.jpg		 Mom School Stuff
Certifications Originals.pdf			 P Guide.pdf
ChatBot						 Tuberculosis
Colab Notebooks					 ZS Data Science 
ContentBlocker


In [6]:
!pwd
!ls

/content
adc.json  drive  sample_data


# Uploading Text File

In [7]:
from google.colab import files
src = list(files.upload().values())[0]
open('test.docx','wb').write(src)

Saving test.docx to test.docx


277101

In [8]:
from google.colab import files
src = list(files.upload().values())[0]
open('vgg16.py','wb').write(src)
import vgg16

Saving vgg16.py to vgg16.py


Using TensorFlow backend.


# Importing Pre Trained Model

In [0]:
import numpy as np
import os
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from vgg16 import VGG16
from keras.models import Model
from keras.preprocessing import image

In [10]:
PATH = os.getcwd()
print(PATH)
data_path = PATH + '/drive/ContentBlocker/Dataset'
print(data_path)

/content
/content/drive/ContentBlocker/Dataset


In [11]:

image_input = Input(shape=(224, 224, 3))

model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

#model.summary()

553467904/553467096 [==============================] - 39s 0us/step


In [0]:

last_layer = model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(3, activation='softmax', name='output')(x)
custom_vgg_model2 = Model(image_input, out)

In [0]:
weights_path = data_path + "/model_weights.h5"

In [0]:
custom_vgg_model2.load_weights(weights_path)

# Applying model on images of docs

In [0]:
#image_path = data_path + '/Train/Close_to_abusive/1.jpg'

In [15]:
from google.colab import files
src = list(files.upload().values())[0]
open('NSFW.docx','wb').write(src)

Saving NSFW.docx to NSFW.docx


3081343

In [0]:
import zipfile

zipfname = 'test.docx'
z = zipfile.ZipFile(zipfname)

In [16]:
!ls
!pwd

adc.json  drive  __pycache__  sample_data  test.docx  vgg16.py
/content


In [17]:
#print list of valid attributes for ZipFile object
print(dir(z))

#print all files in zip archive
all_files = z.namelist()
print(all_files)

['NameToInfo', '_RealGetContents', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_allowZip64', '_comment', '_didModify', '_extract_member', '_filePassed', '_fileRefCnt', '_fpclose', '_lock', '_open_to_write', '_sanitize_windows_name', '_seekable', '_windows_illegal_name_trans_table', '_write_end_record', '_writecheck', '_writing', 'close', 'comment', 'compression', 'debug', 'extract', 'extractall', 'filelist', 'filename', 'fp', 'getinfo', 'infolist', 'mode', 'namelist', 'open', 'printdir', 'pwd', 'read', 'setpassword', 'start_dir', 'testzip', 'write', 'writestr']
['[Content_Types].xml', '_rels/.rels', 'word/_rels/document.xml.rels', 'word/document.xml', 'wo

In [18]:
images = [img_name for img_name in all_files if img_name.split('.')[-1] in ['jpg','jpeg','png']]

print(images)

['word/media/image5.jpeg', 'word/media/image4.jpeg', 'word/media/image3.jpeg', 'word/media/image2.jpeg', 'word/media/image1.jpeg', 'word/media/image6.jpeg']


In [0]:
#get all files in word/media/ directory
#images = list(filter(lambda x: x.startswith('word/media/'), all_files))
#print(images)
#print(len(images))

In [20]:
!pip install paradocx

    100% |████████████████████████████████| 92kB 3.3MB/s 
    100% |████████████████████████████████| 5.8MB 5.7MB/s 
    100% |████████████████████████████████| 51kB 18.3MB/s 
  Running setup.py bdist_wheel for openpack ... - done
  Stored in directory: /root/.cache/pip/wheels/b8/3b/5e/f7205cd1084aa6de3a33014c931c2dab1448cf81441aa7eff8
Successfully built openpack


In [0]:
import os
from IPython.display import Image, display
from keras.preprocessing.image import load_img
from zipfile import ZipFile
from PIL import Image
from io import BytesIO
import PIL
import paradocx
from paradocx import Document
from paradocx.image import ImagePart
import cv2
import os
import tempfile
import zipfile
import shutil

In [22]:
archive = ZipFile(zipfname, 'r')
doc = Document.from_file(zipfname)
filenames = [ig.split('/')[-1] for ig in images]
print(filenames)
done = []

['image5.jpeg', 'image4.jpeg', 'image3.jpeg', 'image2.jpeg', 'image1.jpeg', 'image6.jpeg']


In [23]:
#open an image and save it
for image_name in images:
  '''
  image1 = z.open(image_name).read()
  f = open(temp_image_name,'wb')
  f.write(image1)
  
  #Extract file
  z.extract(image_name, '/content')
  
  # load an image from file
  #image = load_img(extract_path + '/' + image_name.split('/')[-1], target_size=(224, 224))
  image = load_img('/content/'+temp_image_name, target_size=(224, 224))
  
  #print(type(image1))
  '''
  image_data = archive.read(image_name)
  img = cv2.imdecode(np.frombuffer(image_data, np.uint8), 1) 
  image_extension = image_name.split('.')[-1]
  cv2.imwrite(image_name.split('/')[-1],img)
  
  # load an image from file
  image = load_img(image_name.split('/')[-1], target_size=(224, 224))
  #image = load_img('/content/'+'1.'+image_extension, target_size=(224, 224))
  
  
  from keras.preprocessing.image import img_to_array
  # convert the image pixels to a numpy array
  image = img_to_array(image)

  # reshape data for the model
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

  from keras.applications.vgg16 import preprocess_input
  # prepare the image for the VGG model
  image = preprocess_input(image)

  # predict the probability across all output classes
  yhat = custom_vgg_model2.predict(image)
  
  
  
  
  '''
  image1 = ImagePart(doc, '/'+image_name)
  image1.data = gausBlur
  print('start')
  '''

  print(yhat)
  
  #BLUR NON-ABUSIVE IMAGE
  if (yhat[0][2]>=0.5 or yhat[0][1]>=0.5):
    print(image_name.split('/')[-1] + ' is non Abusive')
    gausBlur = cv2.GaussianBlur(np.asarray(img), (45,45),0) 
    for i in range(10):
      gausBlur = cv2.GaussianBlur(gausBlur, (45,45),0)  
    cv2.imwrite(image_name.split('/')[-1],gausBlur)

    tempdir = tempfile.mkdtemp()
    try:
      tempname = os.path.join(tempdir, zipfname)
      with zipfile.ZipFile(zipfname, 'r') as zipread:
          with zipfile.ZipFile(tempname, 'w') as zipwrite:
              for item in zipread.infolist():
                  print(item.filename)
                  if item.filename.split('/')[-1] != image_name.split('/')[-1]:
                      data = zipread.read(item.filename)
                      print('type in zipread:'+str(type(data))+'\ndata:'+str(data))
                      zipwrite.writestr(item, data)
                  else:
                    with open(image_name.split('/')[-1], "rb") as imageFile:
                      f = imageFile.read()
                      data = bytearray(f)
                      print('type in cv2:'+str(type(data))+'\ndata:'+str(data))
                    zipwrite.writestr(item, data)
                    done.append(image_name.split('/')[-1])
      shutil.move(tempname, zipfname)
    finally:
        shutil.rmtree(tempdir)

[[0. 0. 1.]]
[[2.8266233e-01 2.1289577e-05 7.1731633e-01]]
[[1.2698534e-05 1.5757221e-06 9.9998569e-01]]
[[0. 0. 1.]]
[[0. 0. 1.]]
[[0. 0. 1.]]


In [0]:
from google.colab import files
files.download(zipfname)
#files.download('image1.jpeg')
#files.download('1.jpeg')

In [17]:
!ls

adc.json    image1.jpeg  image4.jpeg  imagenet_utils.py  test.docx
blured.jpg  image2.jpeg  image5.jpeg  __pycache__	 vgg16.py
drive	    image3.jpeg  image6.jpeg  sample_data
